#Построение модели для прогнозирования суммы покупки клиента.

### Постановки задачи

Розничная компания хочет понять покупательское поведение клиентов (в частности, сумму покупки) в отношении различных продуктов разных категорий. Они поделились сводкой о покупках различных клиентов для выбранных крупносерийных продуктов за последний месяц. Набор данных также содержит демографические данные клиентов (возраст, пол, семейное положение, город проживания, как долго покупатель живет в этом городе), сведения о продукте (product_id и категория продукта) и общую сумму покупки за последний месяц.

Теперь они хотят построить модель для прогнозирования суммы покупки клиента по различным продуктам, которая поможет им создать персонализированное предложение для клиентов по различным продуктам.

**Определение переменной**

**Подключение библиотек и скриптов**

In [ ]:
import numpy as np
import pandas as pd
import random
import pickle

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score as r2
from sklearn.model_selection import KFold, GridSearchCV

from datetime import datetime

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
matplotlib.rcParams.update({'font.size': 14})

In [ ]:
def evaluate_preds(train_true_values, train_pred_values, test_true_values, test_pred_values):
    print("Train R2:\t" + str(round(r2(train_true_values, train_pred_values), 3)))
    print("Test R2:\t" + str(round(r2(test_true_values, test_pred_values), 3)))
    
    plt.figure(figsize=(18,10))
    
    plt.subplot(121)
    sns.scatterplot(x=train_pred_values, y=train_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Train sample prediction')
    
    plt.subplot(122)
    sns.scatterplot(x=test_pred_values, y=test_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Test sample prediction')

    plt.show()

**Пути к директориям и файлам**

In [ ]:
TRAIN_DATASET_PATH = 'train.csv'
TEST_DATASET_PATH = 'test.csv'

### Загрузка данных <a class='anchor' id='load'>

**Описание датасета**

* **User_ID** - идентификационный номер покупателя
* **Product_ID** - идентификационный номер товара
* **Gender** - пол покупателя
* **Age** - возраст покупателя
* **Occupation** - род деятельности покупателя
* **City_Category** - город проживания
* **Stay_In_Current_City_Years** - как долго покупатель живет в этом городе
* **Marital_Status** - семейное положение покупателя
* **Product_Category_1** - категория товара 1
* **Product_Category_2** - категория товара 2
* **Product_Category_3** - категория товара 3
* **Purchase** - сумма покупок

In [ ]:
train_df = pd.read_csv(TRAIN_DATASET_PATH)
train_df.tail()

FileNotFoundError: ignored

In [ ]:
train_df.info()

In [ ]:
test_df = pd.read_csv(TEST_DATASET_PATH)
test_df.tail()

In [ ]:
test_df.info()

In [ ]:
print('Строк в трейне:', train_df.shape[0])
print('Строк в тесте', test_df.shape[0])

In [ ]:
train_df.shape[1] - 1 == test_df.shape[1]

### Приведение типов

In [ ]:
train_df.dtypes

## 1. EDA  <a class='anchor' id='eda'>
Делаем EDA для:
- Исправления выбросов
- Заполнения NaN
- Идей для генерации новых фич

**Целевая переменная**

In [ ]:
plt.figure(figsize = (16, 8))

train_df['Purchase'].hist(bins=30)
plt.ylabel('Count')
plt.xlabel('Purchase')

plt.title('Target distribution')
plt.show()

In [ ]:
sns.boxplot(train_df["Purchase"])
plt.title("Boxplot of Purchase")
plt.show()

In [ ]:
train_df["Purchase"].skew()

In [ ]:
train_df["Purchase"].kurtosis()

**Качественные переменные**

In [ ]:
train_df.describe()

In [ ]:
train_df.hist(figsize=(16,16), bins=20, grid=False)

**Номинативные переменные**

In [ ]:
train_df.select_dtypes(include='object').columns.tolist()

In [ ]:
train_df['Gender'].value_counts()

In [ ]:
train_df['Age'].value_counts()

In [ ]:
train_df['City_Category'].value_counts()

In [ ]:
train_df['Stay_In_Current_City_Years'].value_counts()

### 2. Обработка выбросов  <a class='anchor' id='outlier'>
Что можно делать с ними?
1. Выкинуть эти данные (только на трейне, на тесте ничего не выкидываем)
2. Заменять выбросы разными методами (медианы, средние значения, np.clip и т.д.)
3. Делать/не делать дополнительную фичу
4. Ничего не делать

Удаляем User_ID и Product_ID

In [ ]:
train_df.drop('User_ID', axis=1, inplace=True)
train_df.drop('Product_ID', axis=1, inplace=True)

**Gender**

In [ ]:
train_df['Gender'].value_counts()

In [ ]:
sns.countplot(train_df['Gender'])
plt.show()

In [ ]:
train_df.loc[(train_df['Gender'] == 'M'), 'Gender'] = 1
train_df.loc[(train_df['Gender'] == 'F'), 'Gender'] = 2
train_df.head()

In [ ]:
train_df['Gender'].value_counts()

**Age**

In [ ]:
train_df['Age'].value_counts()

In [ ]:
sns.countplot(train_df['Age'])
plt.title('Распределение по возрасту')
plt.xlabel('Age')
plt.show()

Как видно из графика, выделяется семь возрастных категорий.

In [ ]:
train_df.loc[train_df['Age'] == '0-17', 'Age'] = 1
train_df.loc[train_df['Age'] == '18-25', 'Age'] = 2
train_df.loc[train_df['Age'] == '26-35', 'Age'] = 3
train_df.loc[train_df['Age'] == '36-45', 'Age'] = 4
train_df.loc[train_df['Age'] == '46-50', 'Age'] = 5
train_df.loc[train_df['Age'] == '51-55', 'Age'] = 6
train_df.loc[train_df['Age'] == '55+', 'Age'] = 7

In [ ]:
train_df['Age'].value_counts()

**Occupation**

In [ ]:
train_df['Occupation'].value_counts()

In [ ]:
sns.countplot(train_df['Occupation'])
plt.title('Распределение по профессии')
plt.xlabel('Occupation')
plt.show()

**City_Category**

In [ ]:
train_df['City_Category'].value_counts()

In [ ]:
sns.countplot(train_df['City_Category'])
plt.title('Распределение по городам')
plt.xlabel('City_Category')
plt.show()

In [ ]:
train_df.loc[(train_df['City_Category'] == 'A'), 'City_Category'] = 1
train_df.loc[(train_df['City_Category'] == 'B'), 'City_Category'] = 2
train_df.loc[(train_df['City_Category'] == 'C'), 'City_Category'] = 3
train_df.head()

In [ ]:
train_df['City_Category'].value_counts()

**Stay_In_Current_City_Years**

In [ ]:
train_df['Stay_In_Current_City_Years'].value_counts()

In [ ]:
sns.countplot(train_df['Stay_In_Current_City_Years'])
plt.title('Распределение по времени проживания в городе')
plt.xlabel('Stay_In_Current_City_Years')
plt.show()

In [ ]:
train_df.loc[(train_df['Stay_In_Current_City_Years'] == '4+'), 'Stay_In_Current_City_Years'] = 4

In [ ]:
train_df.head()

In [ ]:
sns.countplot(train_df['Stay_In_Current_City_Years'])
plt.title('Распределение по времени проживания в городе')
plt.xlabel('Stay_In_Current_City_Years')
plt.show()

**Marital_Status**

In [ ]:
train_df['Marital_Status'].value_counts()

In [ ]:
sns.countplot(train_df['Marital_Status'])
plt.title('Распределение по семейному\n положению покупателя')
plt.xlabel('Marital_Status')
plt.show()

**Product_Category_1**

In [ ]:
train_df['Product_Category_1'].value_counts()

In [ ]:
plt.figure(figsize = (16, 5))
sns.countplot(train_df['Product_Category_1'])
plt.title('Распределение по продукту 1-й категории')
plt.xlabel('Product_Category_1')
plt.show()

**Product_Category_2**

In [ ]:
train_df['Product_Category_2'].value_counts()

In [ ]:
plt.figure(figsize = (16, 5))
sns.countplot(train_df['Product_Category_2'])
plt.title('Распределение по продукту 2-й категории')
plt.xlabel('Product_Category_2')
plt.show()

In [ ]:
train_df['Product_Category_2'].isnull().sum()

In [ ]:
train_df['Product_Category_2'] = train_df['Product_Category_2'].fillna(0)

In [ ]:
train_df['Product_Category_2'] = train_df['Product_Category_2'].astype(int)

In [ ]:
train_df['Product_Category_2'].value_counts()

In [ ]:
plt.figure(figsize = (16, 8))
sns.countplot(train_df['Product_Category_2'])
plt.title('Распределение по продукту 2-й категории')
plt.xlabel('Product_Category_2')
plt.show()

In [ ]:
train_df.head()

**Product_Category_3**

In [ ]:
train_df['Product_Category_3'].value_counts()

In [ ]:
plt.figure(figsize = (16, 5))
sns.countplot(train_df['Product_Category_3'])
plt.title('Распределение по продукту 3-й категории')
plt.xlabel('Product_Category_3')
plt.show()

In [ ]:
train_df['Product_Category_3'].isnull().sum()

In [ ]:
train_df['Product_Category_3'] = train_df['Product_Category_3'].fillna(0)

In [ ]:
train_df['Product_Category_3'] = train_df['Product_Category_3'].astype(int)

In [ ]:
train_df['Product_Category_3'].value_counts()

In [ ]:
plt.figure(figsize = (16, 5))
sns.countplot(train_df['Product_Category_3'])
plt.title('Распределение по продукту 3-й категории')
plt.xlabel('Product_Category_3')
plt.show()

In [ ]:
plt.figure(figsize = (20,8))
sns.heatmap(train_df.corr(),annot=True)
plt.show()

**Создаем функцию для обработки тестовых данных и приводим к виду тренеровочных данных**

In [ ]:
def preprocess_data(df):
    df.loc[(df['Gender'] == 'M'), 'Gender'] = 1
    df.loc[(df['Gender'] == 'F'), 'Gender'] = 2
    df.loc[df['Age'] == '0-17', 'Age'] = 1
    df.loc[df['Age'] == '18-25', 'Age'] = 2
    df.loc[df['Age'] == '26-35', 'Age'] = 3
    df.loc[df['Age'] == '36-45', 'Age'] = 4
    df.loc[df['Age'] == '46-50', 'Age'] = 5
    df.loc[df['Age'] == '51-55', 'Age'] = 6
    df.loc[df['Age'] == '55+', 'Age'] = 7
    df.loc[(df['City_Category'] == 'A'), 'City_Category'] = 1
    df.loc[(df['City_Category'] == 'B'), 'City_Category'] = 2
    df.loc[(df['City_Category'] == 'C'), 'City_Category'] = 3
    df.loc[(df['Stay_In_Current_City_Years'] == '4+'), 'Stay_In_Current_City_Years'] = 4
    df['Product_Category_2'] = df['Product_Category_2'].fillna(0)
    df['Product_Category_2'] = df['Product_Category_2'].astype(int)
    df['Product_Category_3'] = df['Product_Category_3'].fillna(0)
    df['Product_Category_3'] = df['Product_Category_3'].astype(int)
    df.drop('User_ID', axis=1, inplace=True)
    df.drop('Product_ID', axis=1, inplace=True)
    return df


In [ ]:
df = test_df

In [ ]:
preprocess_data(df)

In [ ]:
test_df = df

### **Отбор признаков**

In [ ]:
train_df.columns.tolist()

In [ ]:
target_name = 'Purchase'

### Разбиение на train и test  <a class='anchor' id='split'>

In [ ]:
X = train_df.drop(columns=target_name)
y = train_df[target_name]

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, shuffle=True, random_state=21)

In [ ]:
X_train.shape, X_valid.shape, test_df.shape

In [ ]:
X_train.isna().sum().sum(), X_valid.isna().sum().sum(), test_df.isna().sum().sum()

**Обучение модели**

In [ ]:
rf_model = RandomForestRegressor(random_state=21, criterion='mse')
rf_model.fit(X_train, y_train)

**Оценка модели**

In [ ]:
y_train_preds = rf_model.predict(X_train)
y_test_preds = rf_model.predict(X_valid)

evaluate_preds(y_train, y_train_preds, y_valid, y_test_preds)

**Кросс-валидация**

In [ ]:
cv_score = cross_val_score(rf_model, X_train, y_train, scoring='r2', cv=KFold(n_splits=3, shuffle=True, random_state=21))
cv_score

In [ ]:
cv_score.mean()

**Важность признаков**

In [ ]:
feature_importances = pd.DataFrame(zip(X_train.columns, rf_model.feature_importances_), 
                                   columns=['feature_name', 'importance'])

feature_importances.sort_values(by='importance', ascending=False)

###Прогнозирование на тестовом датасете  <a class='anchor' id='prediction'>

In [ ]:
test_df.shape

In [ ]:
predictions = rf_model.predict(test_df)
predictions

In [ ]:
test_df['predictions'] = predictions

In [ ]:
test_df.head()

In [ ]:
test_df = pd.read_csv(TEST_DATASET_PATH)

In [ ]:
test_df['predictions'] = predictions

In [ ]:
test_df.head()

**Сохранение модели**



In [ ]:
with open('rf_final_model', 'wb') as file:
    pickle.dump(rf_model, file)